In [3]:
%load_ext autoreload
%autoreload 2

import numpy as np
import sys
import pandas as pd

import run_utils

sys.path.append('../') 
import reclab

from reclab.recommenders import SLIM, EASE
from reclab import data_utils

sys.path.append('../tests') 
import utils
import collections


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [81]:
# helper functions for metrics

def compute_PREC_REC_MAP_NDCG_MRR(N, users, recs, test_ratings):
    assert recs.shape[1] >= N
    metrics = ['PREC', 'REC', 'MAP', 'NDCG', 'MRR']
    res = {key:[] for key in metrics}
    test_rating_matrix = np.array(list(test_ratings.keys()))
    for user_id, rec in zip(users, recs):
        test_matrix = test_rating_matrix[test_rating_matrix[:,0]==user_id,1]
        prec, recall, ncdg = precision_recall_ndcg_at_k(N, rec[:N], test_matrix)
        MAP, mrr, ncdg = map_mrr_ndcg(rec[:N], test_matrix)
        res['PREC'].append(prec)
        res['REC'].append(recall)
        res['NDCG'].append(ncdg)
        res['MAP'].append(MAP)
        res['MRR'].append(mrr)
    return {key:np.mean(res[key]) for key in metrics}

## From "A troubling analysis"... 
### https://github.com/MaurizioFD/RecSys2019_DeepLearning_Evaluation/blob/861eafeaba2943458adec22469b147ec492784b6/Conferences/IJCAI/NeuRec_github/eval.py

def precision_recall_ndcg_at_k(k, rankedlist, test_matrix):
    idcg_k = 0
    dcg_k = 0
    n_k = k if len(test_matrix) > k else len(test_matrix)
    if n_k == 0:
        return 0, 0, 0
    for i in range(n_k):
        idcg_k += 1 / np.log2(i + 2)

    b1 = rankedlist
    b2 = test_matrix
    s2 = set(b2)
    hits = [(idx, val) for idx, val in enumerate(b1) if val in s2]
    count = len(hits)

    for c in range(count):
        dcg_k += 1 / np.log2(hits[c][0] + 2)

    return float(count / k), float(count / len(test_matrix)), float(dcg_k / idcg_k)


def map_mrr_ndcg(rankedlist, test_matrix):
    ap = 0
    map = 0
    dcg = 0
    idcg = 0
    mrr = 0
    if len(test_matrix) == 0:
        return 0, 0, 0
    for i in range(len(test_matrix)):
        idcg += 1 / np.log2(i + 2)

    b1 = rankedlist
    b2 = test_matrix
    s2 = set(b2)
    hits = [(idx, val) for idx, val in enumerate(b1) if val in s2]
    count = len(hits)

    for c in range(count):
        ap += (c + 1) / (hits[c][0] + 1)
        dcg += 1 / np.log2(hits[c][0] + 2)

    if count != 0:
        mrr = 1 / (hits[0][0] + 1)

    if count != 0:
        map = ap / count

    return map, mrr, float(dcg / idcg)

# SLIM

In "A troubling analysis" (https://arxiv.org/pdf/1911.07698.pdf) Table 12, SLIM achieves the following results on ML 1M.


| PREC@5   | REC@5   | MAP@5   | NDCG@5   | MRR@5   | PREC@10   | REC@10   | MAP@10   | NDCG@10   |  MRR@10 |
|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|
| 0.4437 |  0.1106 |  0.3692 |  0.1749 |  0.6578 | 0.3813 |  0.1770 |  0.3003 |  0.2321 |  0.667 |


In this paper, the dataset is converted into a implicit dataset, so ratings are either 1 or 0. Evaulation was performed by averaging over five different 80/20 train/test splits. (We will just look at a single split below).

 The [hyperparameters](https://github.com/MaurizioFD/RecSys2019_DeepLearning_Evaluation/blob/861eafeaba2943458adec22469b147ec492784b6/DL_Evaluation_TOIS_Additional_material.pdf) are set as `l1_ratio=1.89e-5` and `alpha=0.049`.
 

In [73]:
users, items, ratings = data_utils.read_dataset('ml-1m')

In [74]:
for key in ratings.keys():
    ratings[key] = (1, ratings[key][1])

In [75]:
all_contexts = collections.OrderedDict([(user_id, np.zeros(0)) for user_id in users])

In [76]:
train_ratings, test_ratings = data_utils.split_ratings(ratings, 0.8, shuffle=True, seed=0)

In [77]:
recommender = SLIM(alpha=0.049, l1_ratio=1.89e-5, seed=0)

In [78]:
recommender.reset(users, items, train_ratings)

In [79]:
recs, _ = recommender.recommend(all_contexts, 10)

In [83]:
for N in [5, 10]:
    res = compute_PREC_REC_MAP_NDCG_MRR(N, users, recs, test_ratings)
    print('@{}:'.format(N), res)

@5: {'PREC': 0.3241721854304636, 'REC': 0.06738210300791221, 'MAP': 0.43651995952906547, 'NDCG': 0.10395269380895251, 'MRR': 0.45423013245033117}
@10: {'PREC': 0.3814238410596027, 'REC': 0.17422638615796424, 'MAP': 0.43883045470150284, 'NDCG': 0.18934677631282193, 'MRR': 0.47939562966466936}


# EASE

In "A troubling analysis" (https://arxiv.org/pdf/1911.07698.pdf), EASE achieves the following results on ML 1M



| PREC@5   | REC@5   | MAP@5   | NDCG@5   | MRR@5   | PREC@10   | REC@10   | MAP@10   | NDCG@10   |  MRR@10 |
|------:|------:|------:|------:|------:|------:|------:|------:|------:|------:|
| 0.4360  | 0.1073  | 0.3608  | 0.1697  | 0.6475 | 0.3745  | 0.1731  | 0.2923  | 0.2259  | 0.65| 
 
In this paper, the dataset is converted into a implicit dataset, so ratings are either 1 or 0. Evaulation was performed by averaging over five different 80/20 train/test splits. (We will just look at a single split below).

The [hyperparameters](https://github.com/MaurizioFD/RecSys2019_DeepLearning_Evaluation/blob/861eafeaba2943458adec22469b147ec492784b6/DL_Evaluation_TOIS_Additional_material.pdf) are set as `lam=1.25e3`


In [84]:
users, items, ratings = data_utils.read_dataset('ml-1m')
for key in ratings.keys():
    ratings[key] = (1, ratings[key][1])
all_contexts = collections.OrderedDict([(user_id, np.zeros(0)) for user_id in users])
train_ratings, test_ratings = data_utils.split_ratings(ratings, 0.8, shuffle=True, seed=0)

In [85]:
recommender = EASE(lam=1.25e3)

In [86]:
recommender.reset(users, items, train_ratings)

/home/sarah/anaconda3/lib/python3.7/site-packages/scipy/sparse/_index.py:126: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [87]:
recs, _ = recommender.recommend(all_contexts, 10)

In [88]:
for N in [5, 10]:
    res = compute_PREC_REC_MAP_NDCG_MRR(N, users, recs, test_ratings)
    print('@{}:'.format(N), res)

@5: {'PREC': 0.32466887417218543, 'REC': 0.06821147096556184, 'MAP': 0.4329348325974981, 'NDCG': 0.10444392654073191, 'MRR': 0.4466197571743929}
@10: {'PREC': 0.38415562913907286, 'REC': 0.17628998806236673, 'MAP': 0.4373985546772671, 'NDCG': 0.19097229149913342, 'MRR': 0.4719510801009146}
